In [4]:
import os
import glob
import itertools

import numpy as np
import pandas as pd

from pptx import Presentation

In [5]:
muni_df = pd.read_csv('other/sdsn/sdsn.gen.csv')

In [6]:
file_names = glob.glob('ppt/*')
file_names = sorted(file_names)

file_names

['ppt/2021.10.23-1ra.pptx',
 'ppt/2021.10.23-2da.pptx',
 'ppt/2021.10.30-1ra.pptx',
 'ppt/2021.10.30-2da.pptx',
 'ppt/2021.11.13-1ra.pptx',
 'ppt/2021.11.13-2da.pptx',
 'ppt/2021.11.20-1ra.pptx',
 'ppt/2021.11.20-2da.pptx',
 'ppt/2021.11.27-1ra.pptx',
 'ppt/2021.11.27-2da.pptx',
 'ppt/2021.12.04-1ra.pptx',
 'ppt/2021.12.04-2da.pptx',
 'ppt/2021.12.18-1ra.pptx',
 'ppt/2021.12.18-2da.pptx',
 'ppt/2022.01.22-1ra.pptx',
 'ppt/2022.01.22-2da.pptx']

In [7]:
def get_tables(doc_data):
    tables = []

    for slide in doc_data.slides:
        slide_tables = []
        for shape in slide.shapes:
            if shape.has_table:
                table_data = []
                for row in shape.table.rows:
                    row_data = []
                    for column in row.cells:
                        row_data.append(column.text_frame.text)

                    if len(row_data) == 2 and '%' in row_data[1]:
                        table_data.append(row_data)

                if table_data:
                    slide_tables.append(table_data)

        if slide_tables:
            slide_tables = itertools.chain(*slide_tables)
            slide_tables = list(slide_tables)
            tables.append(slide_tables)
            
    return tables

def process_tables(doc_tables):
    doc_table = pd.DataFrame([])

    for current_table in doc_tables:
        current_table = pd.DataFrame(current_table)
        current_table.columns = ['municipio', 'porcentaje']

        current_muni_df = muni_df[
            muni_df['municipio'].isin(current_table['municipio'])
        ]
        current_table['departamento'] = current_muni_df['departamento'].value_counts(
            ascending=False
        ).index[0]

        doc_table = pd.concat([doc_table, current_table])
        
    return doc_table

In [8]:
df = pd.DataFrame([])

for file_name in file_names:
    doc_data = Presentation(file_name)
    
    doc_tables = get_tables(doc_data)
    doc_table = process_tables(doc_tables)
    
    doc_date = os.path.basename(file_name).split('-')[0]
    doc_date = pd.to_datetime(doc_date)
    
    doc_table['fecha'] = doc_date
    
    if '1ra' in file_name:
        doc_table['dosis'] = 1
    elif '2da' in file_name:
        doc_table['dosis'] = 2
    else:
        raise('nombre sin formato')
    
    df = pd.concat([df, doc_table])
    
df = df[['fecha', 'departamento', 'municipio', 'dosis', 'porcentaje']]
df.porcentaje = df.porcentaje.str[:-1]

#remove departamento which already exists in merger
# df = df.loc[:,["fecha","municipio","dosis","porcentaje"]]
df

,fecha,departamento,municipio,dosis,porcentaje
0,2021-10-23,La Paz,Aucapata,1,10
1,2021-10-23,La Paz,Caquiaviri,1,11
2,2021-10-23,La Paz,Tito Yupanqui (Parquipujio),1,11
3,2021-10-23,La Paz,Ancoraimes,1,12
4,2021-10-23,La Paz,Calacoto,1,13
...,...,...,...,...,...
30,2022-01-22,Oruro,Machacamarca,2,64
31,2022-01-22,Oruro,Curahuara De Carangas,2,67
32,2022-01-22,Oruro,Oruro,2,78
33,2022-01-22,Oruro,Antequera,2,85


In [9]:
MUNI_PATCH = {
    'Tiawanacu': 'Tiahuanacu',
    'Santiago De Huari': 'Huari',
    'Acacio': 'Acasio',
    'Toco': 'Toko',
    'Shinaota': 'Shinahota',
    'Santivañes': 'Santivañez',
    'Villa Montes': 'Villamontes',
    'Huaracaje': 'Huacaraje',
    'Guayaramerín':'Guayaramerin'
}

In [10]:
df["municipio"] = df["municipio"].replace(MUNI_PATCH)

In [11]:
edu_merger = pd.read_csv("https://gist.githubusercontent.com/pr0nstar/df9829a59214fea0c7f9b70913c0af2c/raw/d872c1c53fc6e1ae7ff64c6fa06756b6ff2ae06a/vac.muni.csv", index_col=0)
edu_merger

,c_ut,municipio,departamento
0,10101,Sucre,Chuquisaca
1,10102,Yotala,Chuquisaca
2,10103,Poroma,Chuquisaca
3,10201,Azurduy,Chuquisaca
4,10202,Tarvita,Chuquisaca
...,...,...,...
334,90401,Nacebe (Santa Rosa Del Abuna),Pando
335,90402,Ingavi (Humaita),Pando
336,90501,Nuevo Manoa (Nueva Esperanza),Pando
337,90502,Villa Nueva (Loma Alta),Pando


In [12]:
df = df.merge(edu_merger, on=['departamento', 'municipio'])
df = df.rename({'c_ut': 'cod_ine',
                'vacunacion': 'porcentaje'},
                 axis=1)
df = df[['fecha', 'cod_ine', 'municipio', 'departamento', 'dosis', 'porcentaje']]
df

,fecha,cod_ine,municipio,departamento,dosis,porcentaje
0,2021-10-23,20503,Aucapata,La Paz,1,10
1,2021-10-23,20503,Aucapata,La Paz,2,9
2,2021-10-30,20503,Aucapata,La Paz,1,10
3,2021-10-30,20503,Aucapata,La Paz,2,9
4,2021-11-13,20503,Aucapata,La Paz,1,13
...,...,...,...,...,...,...
5406,2021-12-04,70706,Camiri,Santa Cruz,2,93
5407,2021-12-18,70706,Camiri,Santa Cruz,1,100
5408,2021-12-18,70706,Camiri,Santa Cruz,2,93
5409,2022-01-22,70706,Camiri,Santa Cruz,1,106


In [13]:
df = df.set_index(list(df.columns[:-1]))
df = df[~df.index.duplicated(keep='last')].reset_index()
df

,fecha,cod_ine,municipio,departamento,dosis,porcentaje
0,2021-10-23,20503,Aucapata,La Paz,1,10
1,2021-10-23,20503,Aucapata,La Paz,2,9
2,2021-10-30,20503,Aucapata,La Paz,1,10
3,2021-10-30,20503,Aucapata,La Paz,2,9
4,2021-11-13,20503,Aucapata,La Paz,1,13
...,...,...,...,...,...,...
5400,2021-12-04,70706,Camiri,Santa Cruz,2,93
5401,2021-12-18,70706,Camiri,Santa Cruz,1,100
5402,2021-12-18,70706,Camiri,Santa Cruz,2,93
5403,2022-01-22,70706,Camiri,Santa Cruz,1,106


In [14]:
# df = df.merge(edu_merger, left_on= "municipio", right_on="municipio")
# df = df.rename({"c_ut":"cod_ine"}, axis=1)
# df = df.loc[:,["fecha","cod_ine","municipio","departamento","dosis","porcentaje"]]
# df

In [15]:
# def get_codines(data, mundict):
#     municipios = pd.read_csv(mundict)
#     # df = pd.read_csv(data, index_col=0)
#     # df = df
#     df['dm'] = df.apply(lambda x: '{} - {}'.format(x['departamento'].lower().strip(), x['municipio'].lower().strip()), axis=1)
#     df['cod_ine'] = df.dm.map(municipios.set_index('dm').cod_ine.to_dict())
#     df['porcentaje'] = df['porcentaje'].apply(lambda x: float(x.replace('%', '')) / 100)
#     return df.drop(columns=['dm'])

# dfx = get_codines(data=df, mundict="other/mundict.csv")

# display(dfx)
# dfx[dfx.municipio=="Santivañez"]

In [16]:
df

,fecha,cod_ine,municipio,departamento,dosis,porcentaje
0,2021-10-23,20503,Aucapata,La Paz,1,10
1,2021-10-23,20503,Aucapata,La Paz,2,9
2,2021-10-30,20503,Aucapata,La Paz,1,10
3,2021-10-30,20503,Aucapata,La Paz,2,9
4,2021-11-13,20503,Aucapata,La Paz,1,13
...,...,...,...,...,...,...
5400,2021-12-04,70706,Camiri,Santa Cruz,2,93
5401,2021-12-18,70706,Camiri,Santa Cruz,1,100
5402,2021-12-18,70706,Camiri,Santa Cruz,2,93
5403,2022-01-22,70706,Camiri,Santa Cruz,1,106


In [17]:
df.to_csv("datos/vacunacion_municipios_bo.csv")

In [26]:
ultima_actualizacion = df.sort_values("fecha", ascending=True).fecha.iloc[-1]
ultima_actualizacion

Timestamp('2022-01-22 00:00:00')

In [30]:
df[df.fecha==ultima_actualizacion].to_csv("datos/vacunacion_municipios_bo_ultima.csv")

In [ ]:
df.to_csv("datos/vacunacion_municipios_bo.csv")